In [4]:
import torch
import torch.nn as nn

# Define Model
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        # Define layers as attributes for easy access
        self.linear1 = nn.Linear(num_features, 3)
        self.linear2 = nn.Linear(3, 1)
        
        # Combine into a sequential network
        self.network = nn.Sequential(
            self.linear1,
            nn.ReLU(),
            self.linear2,
            nn.Sigmoid()
        )

    def forward(self, features):
        return self.network(features)

In [5]:
# Create Dataset

features = torch.rand(10, 5)   # 10 samples, 5 features each
labels = torch.randint(0, 2, (10, 1)).float()  # binary labels

In [6]:
# Create Model

model = Model(features.shape[1])

In [7]:
# Forward Pass
# ---------------------------
outputs = model(features)
print("Model outputs:\n", outputs)

Model outputs:
 tensor([[0.4958],
        [0.4800],
        [0.5142],
        [0.4813],
        [0.4833],
        [0.4787],
        [0.4971],
        [0.4932],
        [0.4806],
        [0.4906]], grad_fn=<SigmoidBackward0>)


In [8]:
# Access Weights

print("Linear1 weights:\n", model.linear1.weight)
print("Linear1 bias:\n", model.linear1.bias)

print("Linear2 weights:\n", model.linear2.weight)
print("Linear2 bias:\n", model.linear2.bias)

Linear1 weights:
 Parameter containing:
tensor([[ 0.3184,  0.3666,  0.2199,  0.0873,  0.0170],
        [-0.3823, -0.2757,  0.0860,  0.1362,  0.1635],
        [-0.1590,  0.0902,  0.2054,  0.3526, -0.1941]], requires_grad=True)
Linear1 bias:
 Parameter containing:
tensor([ 0.1768, -0.2138,  0.1402], requires_grad=True)
Linear2 weights:
 Parameter containing:
tensor([[0.2742, 0.4158, 0.1246]], requires_grad=True)
Linear2 bias:
 Parameter containing:
tensor([-0.2699], requires_grad=True)


In [9]:
# Install torchinfo (only needed once)
!pip install torchinfo

# Import summary
from torchinfo import summary

# Print model summary
summary(model, input_size=(10, 5))


Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [10]:
# ---------------------------
# File: binary_classifier.py
# ---------------------------

import torch
import torch.nn as nn
from torchinfo import summary

# ---------------------------
# Model Definition
# ---------------------------
class BinaryClassifier(nn.Module):
    """
    A configurable binary classifier using fully connected layers.

    Args:
        input_dim (int): Number of input features
        hidden_dim (int): Number of neurons in hidden layer
        dropout_prob (float): Dropout probability for regularization
    """
    def __init__(self, input_dim, hidden_dim=32, dropout_prob=0.2):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.linear2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

# ---------------------------
# Create Large Synthetic Dataset
# ---------------------------
num_samples = 1000   # Large dataset
num_features = 10    # Features per sample

X = torch.rand(num_samples, num_features)      # Random features between 0 and 1
y = torch.randint(0, 2, (num_samples, 1)).float()  # Random binary labels

# ---------------------------
# Create Model
# ---------------------------
model = BinaryClassifier(input_dim=num_features)

# ---------------------------
# Print Model Summary
# ---------------------------
summary(model, input_size=(10, num_features))

# ---------------------------
# Training Function
# ---------------------------
def train_model(model, X, y, lr=0.01, epochs=50, batch_size=32):
    """
    Trains the binary classifier.

    Args:
        model: PyTorch model
        X: Input features (tensor)
        y: Labels (tensor)
        lr: Learning rate
        epochs: Number of epochs
        batch_size: Mini-batch size
    """
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    num_batches = X.shape[0] // batch_size

    for epoch in range(epochs):
        epoch_loss = 0.0

        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            X_batch = X[start:end]
            y_batch = y[start:end]

            # Forward pass
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= num_batches
        if epoch % 5 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# ---------------------------
# Train the Model
# ---------------------------
train_model(model, X, y, lr=0.05, epochs=50, batch_size=32)

# ---------------------------
# Save the Model
# ---------------------------
torch.save(model.state_dict(), "binary_classifier.pth")
print("Model saved as binary_classifier.pth")

# ---------------------------
# Access Model Weights
# ---------------------------
print("Linear1 weights:\n", model.linear1.weight)
print("Linear1 bias:\n", model.linear1.bias)
print("Linear2 weights:\n", model.linear2.weight)
print("Linear2 bias:\n", model.linear2.bias)


Epoch 1/50, Loss: 0.6945
Epoch 6/50, Loss: 0.6941
Epoch 11/50, Loss: 0.6932
Epoch 16/50, Loss: 0.6913
Epoch 21/50, Loss: 0.6902
Epoch 26/50, Loss: 0.6896
Epoch 31/50, Loss: 0.6878
Epoch 36/50, Loss: 0.6891
Epoch 41/50, Loss: 0.6875
Epoch 46/50, Loss: 0.6899
Epoch 50/50, Loss: 0.6888
Model saved as binary_classifier.pth
Linear1 weights:
 Parameter containing:
tensor([[ 0.2905, -0.2864, -0.0035, -0.2161,  0.1460, -0.0055, -0.1835, -0.2442,
          0.0474,  0.0229],
        [-0.2320, -0.0846, -0.0266,  0.2189,  0.0961, -0.2847,  0.1546,  0.3122,
         -0.2869, -0.0170],
        [ 0.0421, -0.2492, -0.1776, -0.0633,  0.1288, -0.1811,  0.0711, -0.1314,
          0.0640, -0.1428],
        [ 0.0958, -0.1274, -0.0552, -0.0707, -0.3074,  0.1011,  0.3016,  0.2917,
          0.0283,  0.1749],
        [ 0.1085, -0.0640,  0.1720,  0.0129,  0.2643,  0.0727, -0.2674,  0.1004,
         -0.1178, -0.2158],
        [ 0.1183, -0.2155,  0.2301, -0.2757,  0.1469,  0.1134,  0.2668, -0.1812,
          0.2

In [11]:
import torch
import torch.nn as nn
from torchinfo import summary

In [12]:
# Model Definition

class BinaryClassifier(nn.Module):
    """
    A configurable binary classifier using fully connected layers.

    Args:
        input_dim (int): Number of input features
        hidden_dim (int): Number of neurons in hidden layer
        dropout_prob (float): Dropout probability for regularization
    """
    def __init__(self, input_dim, hidden_dim=32, dropout_prob=0.2):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.linear2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

In [13]:
# Create Large Synthetic Dataset

num_samples = 1000   # Large dataset
num_features = 10    # Features per sample

X = torch.rand(num_samples, num_features)      # Random features between 0 and 1
y = torch.randint(0, 2, (num_samples, 1)).float()  # Random binary labels


In [14]:
# Create Model

model = BinaryClassifier(input_dim=num_features)


In [15]:
# Print Model Summary

summary(model, input_size=(10, num_features))

Layer (type:depth-idx)                   Output Shape              Param #
BinaryClassifier                         [10, 1]                   --
├─Linear: 1-1                            [10, 32]                  352
├─ReLU: 1-2                              [10, 32]                  --
├─Dropout: 1-3                           [10, 32]                  --
├─Linear: 1-4                            [10, 1]                   33
├─Sigmoid: 1-5                           [10, 1]                   --
Total params: 385
Trainable params: 385
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [16]:
# Training Function

def train_model(model, X, y, lr=0.01, epochs=50, batch_size=32):
    """
    Trains the binary classifier.

    Args:
        model: PyTorch model
        X: Input features (tensor)
        y: Labels (tensor)
        lr: Learning rate
        epochs: Number of epochs
        batch_size: Mini-batch size
    """
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    num_batches = X.shape[0] // batch_size

    for epoch in range(epochs):
        epoch_loss = 0.0

        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            X_batch = X[start:end]
            y_batch = y[start:end]

            # Forward pass
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= num_batches
        if epoch % 5 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")


In [17]:
# Train the Model

train_model(model, X, y, lr=0.05, epochs=50, batch_size=32)

Epoch 1/50, Loss: 0.6974
Epoch 6/50, Loss: 0.6932
Epoch 11/50, Loss: 0.6939
Epoch 16/50, Loss: 0.6939
Epoch 21/50, Loss: 0.6919
Epoch 26/50, Loss: 0.6929
Epoch 31/50, Loss: 0.6921
Epoch 36/50, Loss: 0.6917
Epoch 41/50, Loss: 0.6924
Epoch 46/50, Loss: 0.6928
Epoch 50/50, Loss: 0.6920


In [18]:
# Save the Model

torch.save(model.state_dict(), "binary_classifier.pth")
print("Model saved as binary_classifier.pth")

Model saved as binary_classifier.pth


In [19]:
# Access Model Weights

print("Linear1 weights:\n", model.linear1.weight)
print("Linear1 bias:\n", model.linear1.bias)
print("Linear2 weights:\n", model.linear2.weight)
print("Linear2 bias:\n", model.linear2.bias)

Linear1 weights:
 Parameter containing:
tensor([[-1.2128e-02,  1.6003e-01,  1.9009e-01,  1.5843e-01,  3.0332e-01,
          1.8130e-01, -2.8272e-01, -1.9786e-01,  1.2627e-01, -1.3817e-01],
        [ 9.6744e-03,  2.1849e-01, -1.6622e-02, -1.5785e-01,  2.4932e-01,
          2.6236e-01, -4.3071e-02, -2.0389e-01, -2.1631e-01, -1.0510e-01],
        [-1.1291e-01, -1.3439e-01,  4.8269e-02, -7.6534e-02,  2.5716e-01,
         -1.0736e-01, -1.0541e-01,  9.5992e-02,  3.2954e-02, -3.3875e-01],
        [-2.9754e-01,  1.5669e-01, -1.8798e-01,  1.7776e-01, -2.8656e-01,
          2.3969e-01,  1.7879e-01, -1.6768e-01, -1.1661e-01, -1.6162e-01],
        [ 2.2677e-01,  3.1699e-01, -1.5933e-02, -1.8102e-01, -6.3795e-02,
         -1.3951e-01, -2.6882e-01,  2.5149e-01,  1.7260e-01, -3.0726e-01],
        [-7.4594e-02,  1.4383e-01,  2.9182e-01,  2.2122e-01,  2.6585e-01,
          1.2755e-01, -2.5805e-01,  2.4161e-01, -1.7714e-01,  1.8627e-01],
        [-2.1096e-01, -3.7323e-02, -2.2692e-01, -8.2457e-02, -1.93